In [1]:
import asyncio
import numpy as np
import quaternion

from intrepid_environment.simulator import Simulator

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
sim = Simulator()
await sim.connect()

await sim.rpc("session.restart")
entity = await sim.rpc("map.spawn_uav", {
    "robot_id": 0,
    "position": {
        "x": 0,
        "y": 0,
        "z": 0,
    },
})
await sim.rpc("session.run")

Setup trajectory for drone to follow.

In [4]:
from quadcopter_control.trajectory import Trajectory, TrajectoryPoint

In [5]:
async def sim_step(motors):

    (sim_time, _, drone_state) = await asyncio.gather(
            sim.step(),
            sim.rpc(f"object_{entity}.actuator_control", motors),
            sim.rpc("script.eval", {
                "code": """{
                    position = sim.object.position(ARGS),
                    attitude = sim.object.rotation_quat(ARGS),
                    velocity = sim.object.linear_velocity(ARGS),
                    omega = sim.object.angular_velocity(ARGS),
                    acceleration = sim.object.acceleration(ARGS),
                }""",
                "args": entity,
            }),
        )

    acceleration = np.array([drone_state['acceleration']['y'],
                             drone_state['acceleration']['x'],
                             drone_state['acceleration']['z']], dtype=np.float64)

    position = np.array([drone_state['position']['y'],
                        drone_state['position']['x'],
                        drone_state['position']['z']], dtype=np.float64)

    velocity = np.array([drone_state['velocity']['y'],
                        drone_state['velocity']['x'],
                        drone_state['velocity']['z']], dtype=np.float64)

    attitude = np.quaternion(drone_state['attitude'][3],
                             drone_state['attitude'][0],
                             drone_state['attitude'][1],
                             drone_state['attitude'][2])

    omega = np.array([drone_state['omega']['xy'],
                     drone_state['omega']['zx'],
                     drone_state['omega']['yz']], dtype=np.float64)

    return {
        'time': sim_time,
        'acceleration': acceleration,
        'position': position,
        'velocity': velocity,
        'attitude': attitude,
        'omega': omega
    }

In [6]:
async def get_trajectory_with_initial_position():
    state = await sim_step([0, 0, 0, 0])

    starting_point = TrajectoryPoint(
        time=state['time'],
        acceleration=state['acceleration'],
        attitude=state['attitude'],
        omega=state['omega'],
        position=state['position'],
        velocity=state['velocity'],
    )

    trajectory = Trajectory()
    trajectory.add_point(starting_point)

    return trajectory

In [7]:
trajectory_time_offset = 0
dp = np.zeros(3)

def get_next_trajectory_point(trajectory, sim_time, dt, dp):
    pt = trajectory.next_trajectory_point((sim_time + dt) / 1000.0)
    pt.position += dp
    return pt

In [8]:
from quadcopter_control.controller import QuadcopterController, calculate_drone_moment_of_inertia

In [9]:
DRONE_HEIGHT = 0.028
DRONE_RADIUS = 0.05
DRONE_MASS = 0.041

In [10]:
(Ixx, Iyy, Izz) = calculate_drone_moment_of_inertia(DRONE_HEIGHT, DRONE_RADIUS, DRONE_MASS)
(Ixx, Iyy, Izz)

(2.830366666666667e-05, 2.830366666666667e-05, 5.125000000000001e-05)

## Scenario 0: Zero Gains

Test whether controller runs without controller gains set.

In [11]:
trajectory = await get_trajectory_with_initial_position()

In [12]:
DT_MS = 15.625 # 64 hz

In [ ]:
quadcopter_controller = QuadcopterController(
    DT_MS / 1e3,
    DRONE_MASS,
    Ixx, Iyy, Izz,
    max_tilt_angle=0.52,
    max_ascent_rate=20.0
)

In [14]:
trajectory = await get_trajectory_with_initial_position()

MOTOR_SCALE = 150

async def control_step(state):
    motors = quadcopter_controller.run_control(
        traj_pt=trajectory.next_trajectory_point((state['time'] + DT_MS*1e3)/1e6),
        est_pos=state['position'],
        est_vel=state['velocity'],
        est_omega=state['omega'],
        est_att=state['attitude'],
    ) * MOTOR_SCALE

    return await sim_step(motors.tolist())

## Scenario 1: Altitude Control

Tune gains until drone stays in place. on a desired height.

In [15]:
state = await sim_step([0, 0, 0, 0])
trajectory = await get_trajectory_with_initial_position()

In [16]:
# add 500 datapoints so drone stays in place
sim_time = (await sim_step([0, 0, 0, 0]))['time']
n_episodes = 1000

for i in range(0, n_episodes):
    trajectory.add_point(TrajectoryPoint(
        time=(sim_time + (i*DT_MS*1e3) / 1e6),
        position=np.array([0.0, 0.0, 5.0]),
        velocity=np.zeros(3),
        acceleration=np.zeros(3),
        attitude=np.quaternion(1.0, 0.0, 0.0, 0.0),
        omega=np.array([0.0, 0.0, 0.0])
    ))

In [17]:
async def tune_altitude_control(
        kp_pqr=np.array([95.0, 95.0, 6.0]),
        ki_pos_z=8.0,
        kp_pos_z=5.0,
        kp_vel_z=2.0,
        kappa=2.0,
):

    quadcopter_controller.set_gains(kp_pqr, ki_pos_z, kp_pos_z, kp_vel_z, kappa)

    altitude_log = []
    motor_log = []

    state = await sim_step([0, 0, 0, 0])

    for _ in range(1500):
        traj_pt = trajectory.next_trajectory_point((state['time']+ DT_MS * 1e3) / 1e6)

        motors = quadcopter_controller.run_control(
            traj_pt,
            state['position'],
            state['velocity'],
            state['omega'],
            state['attitude'],
        )

        motors *= MOTOR_SCALE
        motors = np.clip(motors, 45, 100)

        altitude_log.append(state['position'][2])
        motor_log.append(np.mean(motors))

        state = await sim_step(motors.tolist())

    return (altitude_log, motor_log)

In [18]:
import matplotlib.pyplot as plt

In [19]:
(altitude_log, motor_log) = await tune_altitude_control(ki_pos_z=7.0, kp_pos_z=7.0, kp_vel_z=7.0)

plt.plot(altitude_log)
plt.title("Altitude", fontsize=20)
plt.show()

plt.plot(motor_log)
plt.title("Motor cmds", fontsize=20)
plt.show()

181.4560702048897 15.625
337.72203639853797 15.625
493.9901525749823 15.625
650.2604365285201 15.625
806.5328686569043 15.625
962.80746764037 15.625
1119.0842138344124 15.625
1275.363122845222 15.625
1431.644177449275 15.625
1587.927319892415 15.625
1744.212086072072 15.625
1900.4981292522189 15.625
2056.785188817384 15.625
2213.0730599713916 15.625
2369.361576288524 15.625
2525.650605252234 15.625
2681.9400398880025 15.625
2838.2297945898113 15.625
2994.519800249414 15.625
3150.809738957898 15.625
3307.10014309176 15.625
3463.3906090238315 15.625
3619.681159924514 15.625
3775.9717719218393 15.625
3932.262425571968 15.625
4088.5531049720425 15.625
4244.843797064667 15.625
4401.134490981062 15.625
4557.42517775833 15.625
4713.7158497914 15.625
4870.006500686153 15.625
5026.2971249722395 15.625
5182.587717926768 15.625
5338.8782754127 15.625
5495.168793718372 15.625
5651.459269672726 15.625
5807.749754506797 15.625
5964.040127313883 15.625
6120.330459063284 15.625
6276.620737735379 15.62

ClientDisconnectedError: command 492 canceled due to disconnect